In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import datetime
# xlrd is required for reading xls Excel files
import xlrd
import re
import sqlite3
import meteostat as mt
# Use this when displaying markdown in Jupyter Notebooks ( Gemini suggestion )
from IPython.display import display, Markdown
# do random forest regression to predict PV(W) from the other columns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
hourly_nighlty_threshold = 10

# Random Forest Regresion Analysis Of Solar and Weather



In [ ]:
file_enriched_features = "data/processed_data/hourly_solar_copernicus_enriched_data.feather"
file_weather_data = "data/processed_data/hourly_weather_data.feather"
file_solar_data="data/processed_data/daily_solar_data.feather"


In [ ]:
df_enriched_features=pd.read_feather(file_enriched_features)
df_weather_data = pd.read_feather(file_weather_data)
df_solar_data = pd.read_feather(file_solar_data)

In [ ]:
df_solar_per_hour = df_solar_data[['PV(W)','DateTime']].resample('h', on='DateTime').sum()
# 
df_solar_data[df_solar_data['DateTime'].dt.date == datetime.date(2024,12,20)]

In [ ]:
# Show the columns/features for each dataset using pandas
sns.set_theme()
feature_columns = {
    "Enriched Features": df_enriched_features.columns,
    "Weather Data": df_weather_data.columns,
    "Solar Data": df_solar_data.columns,
}
for title, cols in feature_columns.items():
    display(pd.DataFrame({title: cols}))


Convert to hourly date time granularity

This is too see if I can get more accurate readings

In [ ]:
# change all the 3  to an hourly aggregation using resample
print(df_enriched_features.columns.to_list())
df_solar_per_hour = df_solar_data[['PV(W)','DateTime']].resample('h', on='DateTime').agg('mean')
df_weather_per_hour = df_weather_data.resample('h', on='DateTime').agg({'Temperature(C)': 'mean',
                                                                     'Humidity(%)': 'mean',
                                                                     'Sunshine Duration': 'mean',
                                                                     'Condition Code': 'first',})
# 'DateTime', '# Observation period', 'TOA', 'Clear sky GHI', 'Clear sky BHI', 'Clear sky DHI', 'Clear sky BNI', 'GHI', 'BHI', 'DHI', 'BNI', 'Reliability,', 'Time', 'Date', 'POA_Pane_I(W/m^2)', 'POA_Pane_II(W/m^2)', 'POAC_Pane_I(W/m^2)', 'POAC_Pane_II(W/m^2)', 'Power_Pane_I(W)', 'Power_Pane_II(W)', 'Power_ClearSky_Pane_I(W)', 'Power_ClearSky_Pane_II(W)', 'Total_Power_Output(W)', 'Total_Power_ClearSky_Output(W)'
df_enriched_per_hour = df_enriched_features.resample('h', on='DateTime').agg({
    'TOA': 'sum',
    'Clear sky GHI': 'sum',
    'Clear sky BHI': 'sum',
    'Clear sky DHI': 'sum',
    'GHI': 'sum',
    'BHI': 'sum',
    'DHI': 'sum',
    'BNI': 'sum',
    'POA_Pane_I(W/m^2)': 'sum',
    'POA_Pane_II(W/m^2)': 'sum',
    'POAC_Pane_I(W/m^2)': 'sum',
    'POAC_Pane_II(W/m^2)': 'sum',
    'Power_Pane_I(W)': 'sum',
    'Power_Pane_II(W)': 'sum',
    'Power_ClearSky_Pane_I(W)': 'sum',
    'Power_ClearSky_Pane_II(W)': 'sum',
    'Total_Power_Output(W)': 'sum',
    'Total_Power_ClearSky_Output(W)': 'sum'
})

In [ ]:
df_merge_hourly = df_solar_per_hour.merge(df_weather_per_hour, on='DateTime').merge(df_enriched_per_hour, on='DateTime')
df_merge_hourly.reset_index(inplace=True)
# Add WeekOfYear
df_merge_hourly['WeekOfYear'] = df_merge_hourly['DateTime'].dt.isocalendar().week
df_merge_hourly['Month_Sin'] = np.sin(2 * np.pi * (df_merge_hourly['DateTime'].dt.month - 1) / 12)
df_merge_hourly['DayOfYear_Sin'] = np.sin(2 * np.pi * (df_merge_hourly['DateTime'].dt.dayofyear - 1) / 365)
df_merge_hourly['HourOfDay_Sin'] = np.sin(2 * np.pi * (df_merge_hourly['DateTime'].dt.hour) / 24)
# Count number Nan in target Y column
print("Number of NaN in PV(W):", df_merge_hourly['PV(W)'].isna().sum())
# Remove Nan in target Y column
df_merge_hourly = df_merge_hourly.dropna(subset=['PV(W)'])
# Count number of rows after removing NaN
print("Number of NaN  after removing in PV(W):", df_merge_hourly['PV(W)'].isna().sum())
print("Number of rows after removing NaN in PV(W):", len(df_merge_hourly))
print("Dataframe shape after removing NaN in PV(W):", df_merge_hourly.shape)

In [ ]:
# display column list and data types and index
display(pd.DataFrame({"Columns": df_merge_hourly.columns, "Data Types": df_merge_hourly.dtypes}))


## Plot the solar output to determine cutoff points

The objective is to visually examine the output and determine the threshold for night time. It is more accurrate to have a rule that determines that there is no power , and not train the model on this. it skews the accuracy. This was from a gemini prompt to assist in building the model. The Clear SKY GDI shows the output based on clear sky , even at night because of abient light from street lights , and full moon there is still power generation at night. This system can generate up to 300 W on a very bright full moon.

In [ ]:
# plot the hour of day vs max , min , mean of PV(W)
fig, ax = plt.subplots(4,1 , figsize=(15 , 12) )
# flatten ax
ax = ax.flatten()
plt.grid()
# plot hour of day vs min, max , mean of PV(W)
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='PV(W)', estimator='mean', ax=ax[0])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='PV(W)', estimator='max', ax=ax[0])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='PV(W)', estimator='min', ax=ax[0])
# plot the mean line below hourly_nighlty_threshold with dashed line
#sns.lineplot(data=df_merge_hourly[df_merge_hourly['PV(W)'] < hourly_nighlty_threshold], x=df_merge_hourly['DateTime'].dt.hour, y='PV(W)', estimator='mean', ax=ax[0], linestyle='--')
# draw a red vertical line at sunrise and sunset hours (4 and 22)
ax[0].axvline(x=4, color='red', linestyle='--', label='Sunrise')
ax[0].axvline(x=22, color='orange', linestyle='--', label='Sunset')
ax[0].set_title('Hour of Day vs PV(W) Mean, Max, Min')
ax[0].set_xlabel('Hour of Day')
# Same plot for Solar Radiation
ax[1].set_title('Hour of Day vs Sunshine Duration Mean')
ax[1].set_xlabel('Hour of Day')
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Sunshine Duration', estimator='mean', ax=ax[1])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Sunshine Duration', estimator='max', ax=ax[1])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Sunshine Duration', estimator='min', ax=ax[1])
# draw a red vertical line at sunrise and sunset hours (4 and 22)
ax[1].axvline(x=4, color='red', linestyle='--', label='Sunrise')
ax[1].axvline(x=22, color='orange', linestyle='--', label='Sunset')
ax[2].set_title('Hour of Day vs GHI Mean')
ax[2].set_xlabel('Hour of Day')
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='GHI', estimator='mean', ax=ax[2])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='GHI', estimator='max', ax=ax[2])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='GHI', estimator='min', ax=ax[2])
# draw a red vertical line at sunrise and sunset hours (4 and 22)
ax[2].axvline(x=4, color='red', linestyle='--', label='Sunrise')
ax[2].axvline(x=22, color='orange', linestyle='--', label='Sunset')
ax[3].set_title('Hour of Day vs Clear sky GHI Mean')
ax[3].set_xlabel('Hour of Day')
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Clear sky GHI', estimator='mean', ax=ax[3])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Clear sky GHI', estimator='max', ax=ax[3])
sns.lineplot(data=df_merge_hourly, x=df_merge_hourly['DateTime'].dt.hour, y='Clear sky GHI', estimator='min', ax=ax[3])
# draw a red vertical line at sunrise and sunset hours (4 and 22)
ax[3].axvline(x=4, color='red', linestyle='--', label='Sunrise')
ax[3].axvline(x=22, color='orange', linestyle='--', label='Sunset')
plt.tight_layout()
plt.legend()
plt.show()# save the merged dataframe


Look at only hours between 4 and 22 

## Histogram of various solar measures 

This shows a histogram of the various measures solar output. 

In [ ]:
print("Columns:", df_merge_hourly.columns.tolist())
feature_cols = ['Temperature(C)', 'Humidity(%)', 'Sunshine Duration', 'Condition Code', 'Month_Sin', 'DayOfYear_Sin', 'HourOfDay_Sin', 'GHI',
                'Total_Power_Output(W)', 'Power_Pane_I(W)', 'Power_Pane_II(W)','Clear sky GHI',
                'Total_Power_ClearSky_Output(W)','Power_ClearSky_Pane_I(W)','Power_ClearSky_Pane_II(W)']
target_col = 'PV(W)'

In [ ]:
print("Columns:", df_merge_hourly.columns.tolist())#
#feature_cols = ['Temperature(C)', 'Humidity(%)', 'Condition Code', 'Month_Sin', 'DayOfYear_Sin', 'HourOfDay_Sin',
#                'Total_Power_ClearSky_Output(W)','Power_ClearSky_Pane_I(W)','Power_ClearSky_Pane_II(W)']
feature_cols = ['Temperature(C)', 'Humidity(%)', 'Condition Code', 'Month_Sin', 'DayOfYear_Sin', 'HourOfDay_Sin','Total_Power_Output(W)']
target_col = 'PV(W)'
print(f"\nTesting Random Forest Regressor with target: {target_col} and features: {feature_cols}")
# first we filter  out where the output is less than threshold - to avoid night time data , this is based on clearsky GHI
df_daytime = df_merge_hourly['Clear sky GHI'] > 10
# now split into test and training based test and train data sets are every other week
df_daytime_train = df_merge_hourly[(df_merge_hourly['WeekOfYear'] % 2 == 0) & (df_daytime)]
df_daytime_test = df_merge_hourly[(df_merge_hourly['WeekOfYear'] % 2 == 1) & (df_daytime)]
print("Training DataFrame shape:", df_daytime_train.shape)
print("Testing DataFrame shape:", df_daytime_test.shape)
# Create x and y for training and testing
X_train = df_daytime_train[feature_cols]
y_train = df_daytime_train[target_col]
X_test = df_daytime_test[feature_cols]
y_test = df_daytime_test[target_col]


In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42,max_depth=16,min_samples_split=5,min_samples_leaf=2)
# Train the model
print("Fitting model...")
rf_model.fit(X_train, y_train)
# Make predictions
print("Making predictions on test set...")
y_test_pred = rf_model.predict(X_test)
print("Making predictions on test set... Done")
# making predictions on training set
print("Making predictions on training set...")
y_train_pred = rf_model.predict(X_train)
print("Making predictions on training set... Done")
      

__Metrics for Tests 

In [ ]:
# generate 3 main metrics: RMSE, MAE, R2 for both training and testing
# # do random forest regression to predict PV(W) from the other columns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# do random forest regression to predict PV(W) from the other columns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true, y_pred)
    # and normalized based on MAX of y_true - the standard practice for solar prediction
    #normalization_factor = np.max(y_true) 
    normalization_factor = 8400 # based on max PV(W) based on system size
    n_rmse = rmse / normalization_factor
    n_mae = mae / normalization_factor
    # calculate accuracy as 1 - (mae / mean of y_true)
    accuracy_score = 1 - (mae / np.mean(y_true))
    return rmse, mae, r2 , n_rmse, n_mae, accuracy_score
train_rmse, train_mae, train_r2, train_n_rmse, train_n_mae, train_accuracy = calculate_metrics(y_train, y_train_pred)
test_rmse, test_mae, test_r2, test_n_rmse, test_n_mae, test_accuracy = calculate_metrics(y_test, y_test_pred)
# print the metrics , output into a markdown table , include percentages for n_rmse and n_mae and accuracy
mk_string = """| Metric       | Training Set        | Testing Set         |
|--------------|---------------------|---------------------|
| RMSE         | {:.2f}              | {:.2f}              |
| MAE          | {:.2f}              | {:.2f}              |
| R2           | {:.4f}              | {:.4f}              |
| N-RMSE (%)   | {:.4f}              | {:.4f}              |
| N-MAE (%)    | {:.4f}              | {:.4f}              |
| Accuracy (%) | {:.4f}              | {:.4f}              |""".format(
    train_rmse, test_rmse,
    train_mae, test_mae,
    train_r2, test_r2,
    train_n_rmse*100, test_n_rmse*100,
    train_n_mae*100, test_n_mae*100,
    train_accuracy*100, test_accuracy*100
)
from IPython.display import display, Markdown
display(Markdown("### Random Forest Regressor Performance Metrics\n" + mk_string))
#print(f"\nTraining Metrics:\nRMSE: {train_rmse:.2f}, MAE: {train_mae:.2f}, R2: {train_r2:.4f}, N-RMSE: {train_n_rmse:.4f}, N-MAE: {train_n_mae:.4f}, Accuracy: {train_accuracy:.4f}")
#print(f"\nTesting Metrics:\nRMSE: {test_rmse:.2f}, MAE: {test_mae:.2f}, R2: {test_r2:.4f}, N-RMSE: {test_n_rmse:.4f}, N-MAE: {test_n_mae:.4f}, Accuracy: {test_accuracy:.4f}")
# feature importance plot
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_cols, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_cols, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

In [ ]:
# plot residuals
plt.figure(figsize=(10,6))
plt.scatter(y_test, y_test - y_pred, color='blue', label='Test Data', alpha=0.5)
plt.scatter(y_train, y_train - y_train_pred, color='green', label='Train Data', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals vs Actual PV(W)')
plt.xlabel('Actual PV(W)')
plt.ylabel('Residuals (Actual - Predicted)')
plt.legend()
plt.show()


In [ ]:
# plot risduals per hour of day
plt.figure(figsize=(10,6))
plt.scatter(df_daytime_test['DateTime'].dt.hour, y_test - y_pred, color='blue', label='Test Data', alpha=0.5)
plt.scatter(df_daytime_train['DateTime'].dt.hour, y_train - y_train_pred
, color='green', label='Train Data', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals vs Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Residuals (Actual - Predicted)')
plt.legend()
plt.show()

In [ ]:
# Residual histogram
plt.figure(figsize=(10,6))
sns.histplot(y_test - y_pred, color='blue', label='Test Data', kde=True, stat="density", bins=30)
plt.figure(figsize=(10,6))
sns.histplot(y_train - y_train_pred, color='green', label='Train Data', kde=True, stat="density", bins=30)
plt.legend()
plt.show()

In [ ]:
# Add QQ plot for residuals
import scipy.stats as stats
plt.figure(figsize=(10,6))
stats.probplot(y_test - y_pred, dist="norm", plot=plt)
plt.title('QQ Plot of Residuals (Test Data)')
plt.show()


In [ ]:
from xml.parsers.expat import errors
import scipy.stats as stats
# Modelling
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint 
rf_model = RandomForestRegressor(n_estimators=100, random_state=42,max_depth=16,min_samples_split=5,min_samples_leaf=2)
# Train the model
print("Fitting model...")
rf_model.fit(X_train, y_train)
# Make predictions
print("Making predictions on test set...")
y_pred = rf_model.predict(X_test)
print("Making predictions on test set... Done")
# making predictions on training set
y_train_pred = rf_model.predict(X_train)


# Make predictions
print("Making predictions on test set...")
y_pred = rf_model.predict(X_test)
param_dist = {
  'n_estimators': randint(100, 500),
  'max_depth': randint(3, 15),
  'min_samples_split': randint(2, 10),
  'min_samples_leaf': randint(1, 5)
}
rf_random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist,
                                      n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
rf_random_search.fit(X_train, y_train)
print("Best parameters found: ", rf_random_search.best_params_)
# Evaluate the mod
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
# generate 3 main metrics: RMSE, MAE, R2 for both training and testing
# # do random forest regression to predict PV(W) from the other columns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# do random forest regression to predict PV(W) from the other columns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true, y_pred)
    # and normalized based on MAX of y_true - the standard practice for solar prediction
    #normalization_factor = np.max(y_true) 
    normalization_factor = 8400 # based on max PV(W) based on system size
    n_rmse = rmse / normalization_factor
    n_mae = mae / normalization_factor
    # calculate accuracy as 1 - (mae / mean of y_true)
    accuracy_score = 1 - (mae / np.mean(y_true))
    return rmse, mae, r2 , n_rmse, n_mae, accuracy_score
train_rmse, train_mae, train_r2, train_n_rmse, train_n_mae, train_accuracy = calculate_metrics(y_train, y_train_pred)
test_rmse, test_mae, test_r2, test_n_rmse, test_n_mae, test_accuracy = calculate_metrics(y_test, y_test_pred)
# print the metrics , output into a markdown table , include percentages for n_rmse and n_mae and accuracy
mk_string = """| Metric       | Training Set        | Testing Set         |
|--------------|---------------------|---------------------|
| RMSE         | {:.2f}              | {:.2f}              |
| MAE          | {:.2f}              | {:.2f}              |
| R2           | {:.4f}              | {:.4f}              |
| N-RMSE (%)   | {:.4f}              | {:.4f}              |
| N-MAE (%)    | {:.4f}              | {:.4f}              |
| Accuracy (%) | {:.4f}              | {:.4f}              |""".format(
    train_rmse, test_rmse,
    train_mae, test_mae,
    train_r2, test_r2,
    train_n_rmse*100, test_n_rmse*100,
    train_n_mae*100, test_n_mae*100,
    train_accuracy*100, test_accuracy*100
)
from IPython.display import display, Markdown
display(Markdown("### Random Forest Regressor Performance Metrics\n" + mk_string))
#print(f"\nTraining Metrics:\nRMSE: {train_rmse:.2f}, MAE: {train_mae:.2f}, R2: {train_r2:.4f}, N-RMSE: {train_n_rmse:.4f}, N-MAE: {train_n_mae:.4f}, Accuracy: {train_accuracy:.4f}")
#print(f"\nTesting Metrics:\nRMSE: {test_rmse:.2f}, MAE: {test_mae:.2f}, R2: {test_r2:.4f}, N-RMSE: {test_n_rmse:.4f}, N-MAE: {test_n_mae:.4f}, Accuracy: {test_accuracy:.4f}")
# feature importance plot
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_cols, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_cols, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

In [ ]:
# plot residuals
plt.figure(figsize=(10,6))
plt.scatter(y_test, y_test - y_pred, color='blue', label='Test Data', alpha=0.5)
plt.scatter(y_train, y_train - y_train_pred, color='green', label='Train Data', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals vs Actual PV(W)')
plt.xlabel('Actual PV(W)')
plt.ylabel('Residuals (Actual - Predicted)')
plt.legend()
plt.show()


In [ ]:
# plot risduals per hour of day
plt.figure(figsize=(10,6))
plt.scatter(df_daytime_test['DateTime'].dt.hour, y_test - y_pred, color='blue', label='Test Data', alpha=0.5)
plt.scatter(df_daytime_train['DateTime'].dt.hour, y_train - y_train_pred
, color='green', label='Train Data', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals vs Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Residuals (Actual - Predicted)')
plt.legend()
plt.show()